<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# === AUTO-RUN HEARTBEAT (every 60 minutes) ===
import threading, time, IPython

HEARTBEAT_INTERVAL = 60 * 60   # 60 minutes

def heartbeat_loop():
    while True:
        try:
            print("\n❤️  Heartbeat triggered — auto-running keepalive cell...")
            IPython.display.display(IPython.display.Javascript(
                'google.colab.kernel.invokeFunction("keepalive", [], {});'
            ))
        except Exception as e:
            print("⚠️ Heartbeat error:", e)
        time.sleep(HEARTBEAT_INTERVAL)

def start_heartbeat():
    t = threading.Thread(target=heartbeat_loop, daemon=True)
    t.start()
    print("🔥 Auto-run heartbeat started (interval = 60 min).")

# Register a hidden keepalive callback
from google.colab import output
def _keepalive():
    print("⏳ Notebook auto-ran keepalive at", time.ctime())
output.register_callback("keepalive", _keepalive)

start_heartbeat()



❤️  Heartbeat triggered — auto-running keepalive cell...
🔥 Auto-run heartbeat started (interval = 60 min).


<IPython.core.display.Javascript object>

In [24]:
import threading
import time
import shutil
from datetime import datetime

def start_autosave(interval_sec=300, source_path="/content/drive/My Drive/AI-Automation", backup_root="/content/drive/MyDrive/YT_Backups"):
    def autosave_loop():
        while True:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = f"{backup_root}/backup_{timestamp}"

            try:
                shutil.copytree(source_path, backup_path)
                print(f"[AUTOSAVE] Backup created at {backup_path}")
            except Exception as e:
                print(f"[AUTOSAVE ERROR] {e}")

            time.sleep(interval_sec)

    thread = threading.Thread(target=autosave_loop, daemon=True)
    thread.start()
    print(f"[AUTOSAVE] Started autosave every {interval_sec} seconds.")


In [26]:
# FULLY-ROBUST AUTO-PERSIST STARTUP CELL (advanced, single cell)
# - mounts Drive (safe)
# - finds a working Drive root (handles strange mountpoints)
# - restores uploaded sample if found (uses /content/sample_from_upload.mp4 hint)
# - provides: save_to_drive(), start_autosave(), stop_autosave(), git_push_small()
# - autosave fallback to /content/drive_backup if Drive is not writable
# - safe defaults, rotation, size checks, and clear logging

import os, shutil, time, threading, tempfile, subprocess
from pathlib import Path

# ---------- CONFIG ----------
UPLOADED_HINT = Path("/content/sample_from_upload.mp4")   # <-- your session-uploaded file (use as file URL)
PREFERRED_DRIVE_SUBPATH = "AI-Automation"                  # top-level folder in MyDrive to use
MAX_COPY_BYTES = 200 * 1024 * 1024                        # 200 MB
AUTOSAVE_INTERVAL_SEC = 300                               # 5 minutes
BACKUP_ROTATE_KEEP = 12

# ---------- helpers ----------
def _log(*a, **k): print("[AUTO-PERSIST]", *a, **k)

# Standard Colab mount attempt (safe)
def try_mount_drive():
    try:
        from google.colab import drive
    except Exception:
        _log("Not running in Colab (no google.colab). Skipping Drive mount.")
        return False, None
    try:
        # mount to standard location; if already mounted this is idempotent
        drive.mount('/content/drive', force_remount=True)
        _log("drive.mount called.")
    except Exception as e:
        _log("drive.mount raised:", e)

    # detect proper drive root (MyDrive)
    candidates = [
        Path("/content/drive/MyDrive"),
        Path("/content/drive"),   # fallback
        Path("/content/drive_google/MyDrive"),  # some environments
    ]
    for c in candidates:
        if c.exists() and any(c.iterdir()):  # non-empty
            _log("Detected Drive root:", c)
            return True, c
    # if none found, still return mountpoint if exists
    if Path("/content/drive").exists():
        _log("Drive present at /content/drive (but MyDrive not found). Using /content/drive.")
        return True, Path("/content/drive")
    _log("Drive not detected after mount attempt.")
    return False, None

def ensure_drive_paths(drive_root: Path, subpath=PREFERRED_DRIVE_SUBPATH):
    try:
        target = drive_root / subpath
        target.mkdir(parents=True, exist_ok=True)
        (target / "outputs").mkdir(parents=True, exist_ok=True)
        (target / "checkpoints").mkdir(parents=True, exist_ok=True)
        (target / "session_backups").mkdir(parents=True, exist_ok=True)
        return target
    except Exception as e:
        _log("Could not create Drive subpaths:", e)
        return None

def _safe_copy(src: Path, dst: Path, max_bytes=MAX_COPY_BYTES):
    try:
        if not src.exists():
            _log("Source missing:", src)
            return False
        # avoid copying to same file
        try:
            if src.resolve() == dst.resolve():
                _log("Source and destination are identical; skipping copy:", src)
                return True
        except Exception:
            pass
        size = src.stat().st_size
        if size > max_bytes:
            _log("Skipping copy (too large):", src, size)
            return False
        dst.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(str(src), str(dst))
        return True
    except Exception as e:
        _log("Copy failed:", src, dst, e)
        return False

# ---------- main init ----------
_drive_mounted, drive_root = try_mount_drive()
DRIVE_OK = False
DRIVE_BASE = None
if _drive_mounted and drive_root:
    DRIVE_BASE = ensure_drive_paths(drive_root)
    if DRIVE_BASE:
        DRIVE_OK = True
        _log("Drive workspace ready at:", DRIVE_BASE)
    else:
        _log("Drive present but cannot create workspace subpaths (permission/op error). Will fallback.")
else:
    _log("Drive not usable; will fallback to local drive_backup.")

# fallback local backup path
LOCAL_BACKUP_BASE = Path("/content/drive_backup/MyDrive") / PREFERRED_DRIVE_SUBPATH
if not DRIVE_OK:
    LOCAL_BACKUP_BASE.mkdir(parents=True, exist_ok=True)
    (LOCAL_BACKUP_BASE / "outputs").mkdir(parents=True, exist_ok=True)
    (LOCAL_BACKUP_BASE / "session_backups").mkdir(parents=True, exist_ok=True)
    _log("Using local backup base:", LOCAL_BACKUP_BASE)

# ---------- restore uploaded sample (if exists) ----------
_local_sample = None
_drive_sample = None
if UPLOADED_HINT.exists():
    _log("Found uploaded sample hint:", UPLOADED_HINT)
    # copy into /content (if not already there)
    if UPLOADED_HINT.parent != Path("/content"):
        try:
            dst = Path("/content") / UPLOADED_HINT.name
            if _safe_copy(UPLOADED_HINT, dst):
                _local_sample = dst
                _log("Copied uploaded ->", dst)
        except Exception as e:
            _log("Warn: failed to copy into /content:", e)
    else:
        _local_sample = UPLOADED_HINT
        _log("Uploaded already in /content:", _local_sample)
    # also try copy into Drive outputs if available
    if DRIVE_OK:
        drive_sample_path = DRIVE_BASE / "sample.mp4"
        if _safe_copy(UPLOADED_HINT, drive_sample_path):
            _drive_sample = drive_sample_path
            _log("Copied uploaded -> Drive sample:", drive_sample_path)
        else:
            _log("Drive copy failed or skipped for sample.")
    else:
        # fallback: copy to local backup outputs
        local_out = LOCAL_BACKUP_BASE / "outputs" / UPLOADED_HINT.name
        if _safe_copy(UPLOADED_HINT, local_out):
            _log("Copied uploaded -> local backup outputs:", local_out)
else:
    _log("No uploaded sample found at hint path:", UPLOADED_HINT)

# ---------- API: save_to_drive (smart) ----------
def save_to_drive(local_path, dest_name=None):
    local_path = Path(local_path)
    if not local_path.exists():
        raise FileNotFoundError(local_path)
    if local_path.stat().st_size > MAX_COPY_BYTES:
        raise ValueError("File too large to save via helper.")
    ts = int(time.time())
    name = dest_name or f"{local_path.stem}_{ts}{local_path.suffix}"
    if DRIVE_OK:
        dst = DRIVE_BASE / "outputs" / name
        ok = _safe_copy(local_path, dst)
        if ok:
            _log("Saved to Drive:", dst)
            return dst
        else:
            _log("Failed to save to Drive; saving to local backup instead.")
    # fallback
    dst2 = LOCAL_BACKUP_BASE / "outputs" / name
    _safe_copy(local_path, dst2)
    _log("Saved to local backup:", dst2)
    return dst2

# ---------- small git push helper (optional) ----------
def git_push_small(file_path, repo="aicreativeexplorer/YT-Automation", branch="main"):
    token = os.environ.get("GITHUB_TOKEN")
    if not token:
        raise EnvironmentError("Set GITHUB_TOKEN env var in Colab before calling git_push_small().")
    file_path = Path(file_path)
    if not file_path.exists():
        raise FileNotFoundError(file_path)
    if file_path.stat().st_size > 100*1024*1024:
        raise ValueError("File too large for git_push_small.")
    tmp = Path(tempfile.mkdtemp(prefix="kling_git_"))
    clone_url = f"https://{token}@github.com/{repo}.git"
    try:
        _log("Cloning repo to temp...")
        res = subprocess.run(["git","clone","--depth","1","--branch",branch,clone_url,str(tmp)], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if res.returncode != 0:
            _log("Git clone failed:", res.stderr.strip()[:1000])
            return False
        shutil.copy2(str(file_path), str(tmp/file_path.name))
        subprocess.run(["git","config","user.email","colab@local"], cwd=str(tmp))
        subprocess.run(["git","config","user.name","ColabAuto"], cwd=str(tmp))
        subprocess.run(["git","add",file_path.name], cwd=str(tmp))
        subprocess.run(["git","commit","-m",f"Add {file_path.name} via Colab autosave"], cwd=str(tmp))
        push = subprocess.run(["git","push","origin",branch], cwd=str(tmp), stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if push.returncode != 0:
            _log("Git push failed:", push.stderr[:2000])
            return False
        _log("Pushed to GitHub:", repo)
        return True
    finally:
        _log("Git temp dir:", tmp)

# ---------- autosave loop ----------
_autosave_thread = None
_autosave_stop = threading.Event()

def _rotate(base, keep=BACKUP_ROTATE_KEEP):
    try:
        base = Path(base) / "session_backups"
        if not base.exists(): return
        items = sorted([p for p in base.iterdir() if p.is_dir()], key=lambda x: x.stat().st_mtime, reverse=True)
        for old in items[keep:]:
            try:
                shutil.rmtree(old)
                _log("Removed old backup:", old)
            except Exception as e:
                _log("Failed to remove old backup:", old, e)
    except Exception as e:
        _log("Rotate error:", e)

def _autosave_loop(interval_sec=AUTOSAVE_INTERVAL_SEC, paths_to_sync=None, max_bytes=MAX_COPY_BYTES):
    paths_to_sync = paths_to_sync or ["/content"]
    target_base = DRIVE_BASE if DRIVE_OK else LOCAL_BACKUP_BASE
    _log("[autosave] starting. interval=", interval_sec, "target=", target_base)
    while not _autosave_stop.is_set():
        try:
            ts = int(time.time())
            backup_dir = target_base / "session_backups" / f"snapshot_{ts}"
            backup_dir.mkdir(parents=True, exist_ok=True)
            for p in paths_to_sync:
                src = Path(p)
                if not src.exists(): continue
                for f in src.glob("*"):
                    # skip copy of Drive itself
                    if "/content/drive" in str(f): continue
                    try:
                        if f.is_file():
                            _safe_copy(f, backup_dir / f.name, max_bytes)
                        elif f.is_dir():
                            dtarget = backup_dir / f.name
                            dtarget.mkdir(parents=True, exist_ok=True)
                            for sf in f.glob("*"):
                                if sf.is_file():
                                    _safe_copy(sf, dtarget / sf.name, max_bytes)
                    except Exception as e:
                        _log("Autosave copy warning:", f, e)
            _log("[autosave] snapshot ->", backup_dir)
            _rotate(target_base)
        except Exception as e:
            _log("[autosave] loop error:", e)
        _autosave_stop.wait(interval_sec)
    _log("Autosave loop stopped")

def start_autosave(interval_sec=AUTOSAVE_INTERVAL_SEC, paths_to_sync=None):
    global _autosave_thread, _autosave_stop
    if _autosave_thread and _autosave_thread.is_alive():
        _log("Autosave already running.")
        return False
    _autosave_stop.clear()
    _autosave_thread = threading.Thread(target=_autosave_loop, args=(interval_sec, paths_to_sync, MAX_COPY_BYTES), daemon=True)
    _autosave_thread.start()
    _log("[AUTOSAVE] Started every", interval_sec, "seconds. Target:", DRIVE_BASE if DRIVE_OK else LOCAL_BACKUP_BASE)
    return True

def stop_autosave(timeout=5):
    global _autosave_thread, _autosave_stop
    if _autosave_thread and _autosave_thread.is_alive():
        _autosave_stop.set()
        _autosave_thread.join(timeout=timeout)
        _log("Stopped autosave.")
        return True
    _log("No autosave running.")
    return False

# ---------- finish init ----------
_log("INIT COMPLETE.")
_log("Detected sample (local):", str(_local_sample) if _local_sample else "NONE")
_log("Detected sample (drive):", str(_drive_sample) if _drive_sample else "NONE")
_log("Drive usable:", DRIVE_OK, "Drive base:", DRIVE_BASE if DRIVE_OK else LOCAL_BACKUP_BASE)
_log("Uploaded sample path to use as URL:", UPLOADED_HINT)

# start autosave automatically but only if we have a target
start_autosave()

# Expose variables for interactive use
__AUTO_PERSIST_META__ = dict(
    DRIVE_OK=DRIVE_OK,
    DRIVE_BASE=str(DRIVE_BASE if DRIVE_OK else LOCAL_BACKUP_BASE),
    LOCAL_SAMPLE=str(_local_sample) if _local_sample else "",
    DRIVE_SAMPLE=str(_drive_sample) if _drive_sample else "",
    UPLOADED_HINT=str(UPLOADED_HINT)
)

_log("Done. Use save_to_drive(path), git_push_small(path) (with GITHUB_TOKEN), stop_autosave() if needed.")


Mounted at /content/drive
[AUTO-PERSIST] drive.mount called.
[AUTO-PERSIST] Detected Drive root: /content/drive/MyDrive
[AUTO-PERSIST] Drive workspace ready at: /content/drive/MyDrive/AI-Automation
[AUTO-PERSIST] No uploaded sample found at hint path: /content/sample_from_upload.mp4
[AUTO-PERSIST] INIT COMPLETE.
[AUTO-PERSIST] Detected sample (local): NONE
[AUTO-PERSIST] Detected sample (drive): NONE
[AUTO-PERSIST] Drive usable: True Drive base: /content/drive/MyDrive/AI-Automation
[AUTO-PERSIST] Uploaded sample path to use as URL: /content/sample_from_upload.mp4
[AUTO-PERSIST] [autosave] starting. interval= 300 target= /content/drive/MyDrive/AI-Automation
[AUTO-PERSIST] [AUTOSAVE] Started every 300 seconds. Target: /content/drive/MyDrive/AI-Automation
[AUTO-PERSIST] Done. Use save_to_drive(path), git_push_small(path) (with GITHUB_TOKEN), stop_autosave() if needed.


In [27]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764054196
Mounted at /content/drive


In [28]:
%%bash
# 1) Inspect what's currently in /content/drive
echo "Contents of /content/drive BEFORE fix:"
ls -la /content/drive || true
echo "----"

# 2) If non-empty, move it to a safe backup folder instead of deleting
mkdir -p /content/drive_backup || true
if [ "$(ls -A /content/drive 2>/dev/null)" ]; then
  echo "Moving existing /content/drive/* -> /content/drive_backup/"
  mv /content/drive/* /content/drive_backup/ 2>/dev/null || true
  echo "Moved. Backup dir: /content/drive_backup/"
else
  echo "/content/drive is already empty."
fi
echo "----"

# 3) Ensure mountpoint dir exists and is empty
rm -rf /content/drive
mkdir -p /content/drive


Contents of /content/drive BEFORE fix:
total 16
dr-x------ 4 root root 4096 Nov 25 07:03 .Encrypted
drwx------ 2 root root 4096 Nov 25 07:03 MyDrive
dr-x------ 2 root root 4096 Nov 25 07:03 .shortcut-targets-by-id
drwx------ 5 root root 4096 Nov 25 07:03 .Trash-0
----
Moving existing /content/drive/* -> /content/drive_backup/
Moved. Backup dir: /content/drive_backup/
----


rm: cannot remove '/content/drive/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.Trash-0': Directory not empty
rm: cannot remove '/content/drive/.Encrypted/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.Encrypted/.shortcut-targets-by-id': Operation canceled


In [29]:
%%bash
SRC="/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

if [ -f "$SRC" ]; then
  mkdir -p /content/drive/My Drive/AI-/My Drive/AI-Automation /content/drive/My Drive/AI-Automation/sample.mp4
  cp "$SRC" "/content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"
  echo "Copied uploaded file to Drive and /content."
else
  echo "WARNING: Uploaded file missing at $SRC"
fi


In [30]:
# CONFIG — edit only if you moved paths
REPO_DIR = "/content/YT-Automation"
NOTEBOOK_NAME = "YT-Automation.ipynb"
UPLOADED_VIDEO = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"  # your uploaded demo
DRIVE_TOKEN_PATH = "/content/drive/My Drive/AI-Automation/hf_token.txt"  # put HF token here
OUTPUT_DRIVE_FOLDER = "/content/drive/My Drive/AI-Automation/outputs/stitched"
CHECKPOINT_DIR = "/content/drive/My Drive/AI-Automation/checkpoints"
SVD_VERSION = "svd"  # 'svd' (open) or 'svd-xt-1-1' (better but gated)
USE_AUTO_PUSH = False  # no auto-push by default
print("CONFIG OK")


CONFIG OK


In [31]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Auto-login to HuggingFace if token present in Drive
import os
if os.path.exists(DRIVE_TOKEN_PATH):
    from huggingface_hub import login
    with open(DRIVE_TOKEN_PATH,'r') as f:
        token = f.read().strip()
    login(token=token)
    print("Logged into HuggingFace from Drive token.")
else:
    print("No HF token at", DRIVE_TOKEN_PATH, "- you'll be asked if downloading gated models.")


Mounted at /content/drive
No HF token at /content/drive/My Drive/AI-Automation/hf_token.txt - you'll be asked if downloading gated models.


In [37]:
# Full backend replacement: write Flask app, start server in background
# Paste & run this cell in Colab.

import os, textwrap, subprocess, time, threading
from pathlib import Path

# 1) Ensure Flask is installed
print("Installing Flask (if missing)...")
subprocess.run(["python", "-m", "pip", "install", "-q", "Flask"], check=False)

# 2) Write the Flask backend file
backend_path = Path("/content/klingai_flask_backend.py")
backend_code = r'''
import os, json, time, uuid, shutil, threading, subprocess
from pathlib import Path
from flask import Flask, request, jsonify, send_file, abort

# ---------- CONFIG ----------
UPLOAD_ROOT = Path("/content/uploads")
JOB_ROOT = Path("/tmp/klingai_jobs")
OUTPUT_LOCAL = Path("/content/outputs")
# try Drive output if mounted
DRIVE_OUTPUTS = Path("/content/drive/My Drive/AI-Automation/outputs")
if DRIVE_OUTPUTS.exists():
    OUTPUT_ROOT = DRIVE_OUTPUTS
else:
    OUTPUT_ROOT = OUTPUT_LOCAL

UPLOAD_ROOT.mkdir(parents=True, exist_ok=True)
JOB_ROOT.mkdir(parents=True, exist_ok=True)
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
OUTPUT_LOCAL.mkdir(parents=True, exist_ok=True)

# Developer-provided uploaded path (session upload hint)
UPLOADED_HINT = Path(r"/mnt/data/YT_Automation (1).ipynb")

# Job store
_jobs = {}
_jobs_lock = threading.Lock()

# Worker thread pool (just spawn per job for simplicity)
def _create_simulated_mp4(out_path: Path, text="Simulated output", duration=2):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    txt = str(text).replace("'", "").replace("\n"," ")[:200]
    cmd = [
        "ffmpeg","-y",
        "-f","lavfi","-i",f"color=size=720x1280:rate=6:color=0x101018",
        "-t", str(max(1,int(duration))),
        "-vf", f"drawtext=text='{txt}':fontsize=28:fontcolor=white:x=20:y=40",
        str(out_path)
    ]
    try:
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
        return True, ""
    except Exception as e:
        return False, str(e)

def _try_call_sample(cfg, out_path:Path):
    """
    Attempt to call a sample() function available in the global session.
    This works only if the notebook/session has defined sample() and necessary libs.
    """
    try:
        if 'sample' in globals() and callable(globals()['sample']):
            kwargs = dict(
                input_path=cfg.get("conditioning_image",""),
                resize_image=True,
                num_frames=cfg.get("num_frames", 14),
                num_steps=cfg.get("num_steps", 30),
                seed=cfg.get("seed", "random"),
                decoding_t=2,
                fps_id=6,
                motion_bucket_id=127,
                cond_aug=0.02,
                device='cuda' if ('torch' in globals() and __import__('torch').cuda.is_available()) else 'cpu',
                skip_filter=True
            )
            out = globals()['sample'](**kwargs)
            # If sample returned a path or list, try to use it
            if isinstance(out, (list, tuple)) and out:
                candidate = Path(out[0])
                if candidate.exists():
                    shutil.copy2(str(candidate), str(out_path))
                    return True, "sample() produced output"
            elif isinstance(out, str):
                candidate = Path(out)
                if candidate.exists():
                    shutil.copy2(str(candidate), str(out_path))
                    return True, "sample() produced output"
    except Exception as e:
        return False, f"sample() call failed: {e}"
    return False, "no sample() available"

def _process_job(jobid):
    with _jobs_lock:
        job = _jobs.get(jobid)
        if not job:
            return
        job['status'] = 'running'
        job['logs'].append('Job started')
    jobdir = Path(job['jobdir'])
    out_path = jobdir / f"{jobid}.mp4"
    cfg = job.get('cfg', {})
    # 1) if user uploaded a file for this job, use it as conditioning / copy to output (fast path)
    user_file = job.get('uploaded_file')
    used_sample = False
    if user_file:
        upath = Path(user_file)
        if upath.exists():
            # attempt to remux or copy to out_path
            try:
                # remux to mp4 with libx264 to ensure compatibility
                cmd = ["ffmpeg","-y","-i", str(upath), "-c:v","libx264","-pix_fmt","yuv420p","-crf","18", str(out_path)]
                subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
                used_sample = True
                _msg = "Used uploaded file as output"
                with _jobs_lock:
                    job['status'] = 'done'
                    job['progress'] = 100
                    job['logs'].append(_msg)
                    job['output'] = str(out_path)
                # copy to OUTPUT_ROOT
                try:
                    dst = OUTPUT_ROOT / out_path.name
                    shutil.copy2(str(out_path), str(dst))
                except Exception:
                    pass
                return
            except Exception as e:
                with _jobs_lock:
                    job['logs'].append("Failed to remux uploaded file: " + str(e))
    # 2) attempt to call sample() if present
    try:
        ok, info = _try_call_sample(cfg, out_path)
        if ok:
            with _jobs_lock:
                job['status'] = 'done'
                job['progress'] = 100
                job['logs'].append(str(info))
                job['output'] = str(out_path)
            try:
                dst = OUTPUT_ROOT / out_path.name
                shutil.copy2(str(out_path), str(dst))
            except Exception:
                pass
            return
        else:
            with _jobs_lock:
                job['logs'].append("sample() not used: " + str(info))
    except Exception as e:
        with _jobs_lock:
            job['logs'].append("sample() call error: " + str(e))

    # 3) if we have an explicit conditioning image/video path in cfg, and it exists, try simple transform (remux -> ensure mp4)
    cond = cfg.get('conditioning_image') or cfg.get('input_path') or cfg.get('conditioning_video')
    if cond:
        condp = Path(cond)
        if condp.exists():
            try:
                cmd = ["ffmpeg","-y","-i", str(condp), "-c:v","libx264","-pix_fmt","yuv420p","-crf","18", str(out_path)]
                subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
                with _jobs_lock:
                    job['status'] = 'done'
                    job['progress'] = 100
                    job['logs'].append("Transcoded conditioning file to output")
                    job['output'] = str(out_path)
                try:
                    dst = OUTPUT_ROOT / out_path.name
                    shutil.copy2(str(out_path), str(dst))
                except Exception:
                    pass
                return
            except Exception as e:
                with _jobs_lock:
                    job['logs'].append("Failed transcode of conditioning file: " + str(e))

    # 4) fallback: create a simulated MP4 with drawtext using the prompt
    try:
        prompt = cfg.get('prompt') or "Tiny AI short."
        duration = int(cfg.get('duration', 2))
        ok, err = _create_simulated_mp4(out_path, prompt, duration)
        if ok:
            with _jobs_lock:
                job['status'] = 'done'
                job['progress'] = 100
                job['logs'].append("Created simulated mp4")
                job['output'] = str(out_path)
            try:
                dst = OUTPUT_ROOT / out_path.name
                shutil.copy2(str(out_path), str(dst))
            except Exception:
                pass
            return
        else:
            with _jobs_lock:
                job['status'] = 'error'
                job['logs'].append("Simulated mp4 creation failed: " + str(err))
            return
    except Exception as e:
        with _jobs_lock:
            job['status'] = 'error'
            job['logs'].append("Unexpected processing error: " + str(e))
        return

# Flask app
app = Flask(__name__)

@app.route("/", methods=["GET"])
def root():
    return jsonify({"ok": True, "msg":"KlingAI Flask backend alive"}), 200

@app.route("/api/generate", methods=["POST"])
def api_generate():
    try:
        prompt = request.form.get("prompt", "") or request.values.get("prompt","")
        mode = request.form.get("mode", "TEXT")
        duration = request.form.get("duration", request.form.get("dur","2"))
        num_frames = int(request.form.get("num_frames", request.form.get("frames", 25)))
        seed = request.form.get("seed", "random")
        # create job
        jobid = "job-" + uuid.uuid4().hex[:12]
        jobdir = JOB_ROOT / jobid
        jobdir.mkdir(parents=True, exist_ok=True)
        cfg = {
            "prompt": prompt,
            "mode": mode,
            "duration": int(duration) if str(duration).isdigit() else 2,
            "num_frames": num_frames,
            "seed": seed,
            "created_at": int(time.time())
        }
        # if file uploaded with request, save it
        uploaded_file_path = None
        if "file" in request.files:
            f = request.files["file"]
            fname = f.filename or f"{jobid}_upload"
            dst = jobdir / fname
            f.save(str(dst))
            uploaded_file_path = str(dst)
            cfg['conditioning_image'] = str(dst)
        else:
            # fallback: if developer-provided hint exists and is a media file, use as conditioning
            if UPLOADED_HINT.exists() and UPLOADED_HINT.is_file():
                # only accept if extension looks like media
                if UPLOADED_HINT.suffix.lower() in [".mp4",".mov",".mkv",".avi",".webm",".gif"]:
                    cfg['conditioning_image'] = str(UPLOADED_HINT)
                else:
                    # we still store hint info for logs, but won't use as media
                    cfg['hint_note'] = str(UPLOADED_HINT)

        job_record = {
            "jobId": jobid,
            "jobdir": str(jobdir),
            "cfg": cfg,
            "status": "queued",
            "progress": 0,
            "logs": ["Job queued"],
            "uploaded_file": uploaded_file_path,
            "output": None
        }
        with _jobs_lock:
            _jobs[jobid] = job_record

        # start background processing thread for this job
        t = threading.Thread(target=_process_job, args=(jobid,), daemon=True)
        t.start()

        return jsonify({"jobId": jobid}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/api/job/<jobid>", methods=["GET"])
def api_job(jobid):
    with _jobs_lock:
        job = _jobs.get(jobid)
        if not job:
            return jsonify({"error":"job not found"}), 404
        # return safe fields
        return jsonify({
            "jobId": job.get("jobId"),
            "status": job.get("status"),
            "progress": job.get("progress"),
            "logs": job.get("logs")[-20:],
            "outputUrl": f"/api/output/{jobid}" if job.get("output") else None
        }), 200

@app.route("/api/output/<jobid>", methods=["GET"])
def api_output(jobid):
    with _jobs_lock:
        job = _jobs.get(jobid)
        if not job:
            return jsonify({"error":"job not found"}), 404
        out = job.get("output")
        if not out:
            return jsonify({"error":"no output yet"}), 404
        p = Path(out)
        if not p.exists():
            return jsonify({"error":"output missing"}), 404
        # serve file
        try:
            return send_file(str(p), mimetype="video/mp4", as_attachment=False)
        except Exception as e:
            return jsonify({"error":"send_file failed: "+str(e)}), 500

if __name__ == "__main__":
    # debug run
    app.run(host="0.0.0.0", port=7860, threaded=True)
'''
backend_path.write_text(backend_code)
print("Wrote backend to", backend_path)

# 3) Start the backend in the background using nohup so it persists in Colab
print("Starting backend in background (nohup). Output -> /content/klingai_backend.log")
cmd = f"nohup python {str(backend_path)} > /content/klingai_backend.log 2>&1 & echo $!"
proc = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
pid = proc.stdout.strip()
print("Backend launch command output (pid or blank):", pid)
time.sleep(1.2)
# show last few lines of log to confirm
if Path("/content/klingai_backend.log").exists():
    print("--- backend log (last 40 lines) ---")
    print("\n".join(Path("/content/klingai_backend.log").read_text().splitlines()[-40:]))
else:
    print("No backend log yet. If process started, check /content/klingai_backend.log later.")

print("\nBackend should be listening on 0.0.0.0:7860 in this Colab runtime.")
print("If using ngrok, open a tunnel to port 7860 and use the public URL for the React UI.")
print("Example to create an ngrok tunnel (if pyngrok installed and authtoken configured):")
print("  from pyngrok import ngrok; ngrok.connect(7860, 'http')")
print("API endpoints: /api/generate (POST), /api/job/<jobId> (GET), /api/output/<jobId> (GET)")
print("Developer hint path used as uploaded-file hint:", "/mnt/data/YT_Automation (1).ipynb")


Installing Flask (if missing)...
Wrote backend to /content/klingai_flask_backend.py
Starting backend in background (nohup). Output -> /content/klingai_backend.log
Backend launch command output (pid or blank): 31894
--- backend log (last 40 lines) ---
 * Serving Flask app 'klingai_flask_backend'
 * Debug mode: off
Address already in use
Port 7860 is in use by another program. Either identify and stop that program, or start the server with a different port.

Backend should be listening on 0.0.0.0:7860 in this Colab runtime.
If using ngrok, open a tunnel to port 7860 and use the public URL for the React UI.
Example to create an ngrok tunnel (if pyngrok installed and authtoken configured):
  from pyngrok import ngrok; ngrok.connect(7860, 'http')
API endpoints: /api/generate (POST), /api/job/<jobId> (GET), /api/output/<jobId> (GET)
Developer hint path used as uploaded-file hint: /mnt/data/YT_Automation (1).ipynb


In [33]:
from pyngrok import ngrok, conf
import time, os

token = "35mgc5Udwe6mQ6G8HekXP3Rla9x_7zjHMiABfpRhE1j3aWAm5"

print("Setting ngrok authtoken...")
ngrok.set_auth_token(token)
conf.get_default().auth_token = token
print("Done.")


Setting ngrok authtoken...
Done.


In [34]:
# Step A: start pyngrok tunnel and print public URL
# Run this in a Python cell in Colab.

# install pyngrok if needed
import os, time
try:
    from pyngrok import ngrok
except Exception:
    print("Installing pyngrok...")
    os.system("python -m pip install -q pyngrok")
    time.sleep(0.5)
    from pyngrok import ngrok

# create tunnel to port 7860 (your Flask backend)
print("Opening ngrok tunnel to localhost:7860 ...")
tunnel = ngrok.connect(7860, "http")
print("NGROK TUNNEL OPENED ->", tunnel.public_url)
print("If you get an auth error, run: from pyngrok import ngrok; ngrok.set_auth_token('YOUR_TOKEN')")


Opening ngrok tunnel to localhost:7860 ...
NGROK TUNNEL OPENED -> https://advertizable-interpenetratively-abbie.ngrok-free.dev
If you get an auth error, run: from pyngrok import ngrok; ngrok.set_auth_token('YOUR_TOKEN')


In [35]:
# Run this in a Python cell (Colab). It opens ngrok -> 7860 and prints the public URL.
import os, time
from getpass import getpass

# install pyngrok if missing
try:
    from pyngrok import ngrok, conf
except Exception:
    print("Installing pyngrok...")
    os.system("python -m pip install -q pyngrok")
    time.sleep(0.5)
    from pyngrok import ngrok, conf

# If there's no auth token configured, prompt safely (only if needed)
current_token = conf.get_default().auth_token
if not current_token:
    print("No ngrok auth token found in config. If you have one paste it now; otherwise press Enter to abort.")
    token = getpass("ngrok authtoken (paste only the token string, NOT commands): ").strip()
    if not token:
        raise SystemExit("No token provided. Provide a valid ngrok auth token and re-run.")
    conf.get_default().auth_token = token
    try:
        ngrok.set_auth_token(token)
    except Exception:
        pass

# Close any existing tunnels (cleanup)
try:
    for t in ngrok.get_tunnels():
        try:
            ngrok.disconnect(t.public_url)
        except Exception:
            pass
except Exception:
    pass

# Try to open tunnel
print("Opening ngrok tunnel to 7860 ...")
try:
    t = ngrok.connect(7860, "http")
    print("NGROK PUBLIC URL ->", t.public_url)
except Exception as e:
    print("Failed to open ngrok tunnel:", repr(e))
    # show pyngrok logs if available
    log_path = "/tmp/pyngrok.log"
    if os.path.exists(log_path):
        print("--- pyngrok log tail ---")
        print(open(log_path,"r",encoding="utf8",errors="ignore").read().splitlines()[-40:])
    raise

# show tunnels list & quick local sanity curl
print("\npyngrok.get_tunnels():", ngrok.get_tunnels())
print("\nQuick backend root check (server must be running in this Colab):")
import subprocess
subprocess.run(["bash","-lc", "curl -sS --max-time 5 {}/ || echo 'local public root failed'".format(t.public_url)])


Opening ngrok tunnel to 7860 ...
NGROK PUBLIC URL -> https://advertizable-interpenetratively-abbie.ngrok-free.dev

pyngrok.get_tunnels(): [<NgrokTunnel: "https://advertizable-interpenetratively-abbie.ngrok-free.dev" -> "http://localhost:7860">]

Quick backend root check (server must be running in this Colab):


CompletedProcess(args=['bash', '-lc', "curl -sS --max-time 5 https://advertizable-interpenetratively-abbie.ngrok-free.dev/ || echo 'local public root failed'"], returncode=0)

In [39]:
%%bash
NGROK="https://advertizable-interpenetratively-abbie.ngrok-free.dev"
FILE="/content/sample_from_upload.mp4"

echo "Using sample:" $FILE
if [ ! -f "$FILE" ]; then
  echo "ERROR: sample missing: $FILE"
  exit 1
fi

echo "Posting to $NGROK/api/generate ..."
curl -s -X POST "$NGROK/api/generate" \
  -F "prompt=9:16 tiny mechanical fox exploring a sunlit garden, cinematic" \
  -F "mode=IMAGE" \
  -F "duration=4" \
  -F "file=@${FILE}" \
  -o /tmp/generate_resp.json

echo "=== generate response ==="
cat /tmp/generate_resp.json
echo
echo "If you got a jobId (e.g. {\"jobId\":\"job-abc123\"}), paste it here."


Using sample: /content/sample_from_upload.mp4
ERROR: sample missing: /content/sample_from_upload.mp4


CalledProcessError: Command 'b'NGROK="https://advertizable-interpenetratively-abbie.ngrok-free.dev"\nFILE="/content/sample_from_upload.mp4"\n\necho "Using sample:" $FILE\nif [ ! -f "$FILE" ]; then\n  echo "ERROR: sample missing: $FILE"\n  exit 1\nfi\n\necho "Posting to $NGROK/api/generate ..."\ncurl -s -X POST "$NGROK/api/generate" \\\n  -F "prompt=9:16 tiny mechanical fox exploring a sunlit garden, cinematic" \\\n  -F "mode=IMAGE" \\\n  -F "duration=4" \\\n  -F "file=@${FILE}" \\\n  -o /tmp/generate_resp.json\n\necho "=== generate response ==="\ncat /tmp/generate_resp.json\necho\necho "If you got a jobId (e.g. {\\"jobId\\":\\"job-abc123\\"}), paste it here."\n'' returned non-zero exit status 1.

In [ ]:
import time, requests, os
NGROK = "https://advertizable-interpenetratively-abbie.ngrok-free.dev"
JOB_ID = "job-19c6c1f1f2a8"

poll_url = f"{NGROK}/api/job/{JOB_ID}"
print("Polling:", poll_url)

while True:
    try:
        r = requests.get(poll_url, timeout=10)
        data = r.json()
        print("STATUS:", data.get("status"), "| PROGRESS:", data.get("progress"))
        print("LOGS:", data.get("logs")[-3:])

        if data.get("status") == "done":
            print("\n🎉 Job finished!")
            output_url = f"{NGROK}{data['outputUrl']}"
            print("Output URL:", output_url)
            break

        if data.get("status") == "error":
            print("\n❌ Error in job:", data)
            break

    except Exception as e:
        print("Poll error:", e)

    time.sleep(2)


In [ ]:
import requests

OUTPUT_URL = "https://advertizable-interpenetratively-abbie.ngrok-free.dev/api/output/job-19c6c1f1f2a8"
OUTPUT_FILE = "/content/generated_output.mp4"

r = requests.get(OUTPUT_URL, stream=True)
with open(OUTPUT_FILE, "wb") as f:
    for chunk in r.iter_content(chunk_size=1024 * 1024):
        if chunk:
            f.write(chunk)

print("Saved video to:", OUTPUT_FILE)


In [ ]:
%%bash
python -m pip install -q flask-cors

In [ ]:
# STEP 2 — Add CORS support to backend

import re
from pathlib import Path

backend_path = Path("/content/klingai_flask_backend.py")
code = backend_path.read_text()

# Insert CORS import
code = code.replace(
    "from flask import Flask, request, jsonify, send_file, abort",
    "from flask import Flask, request, jsonify, send_file, abort\nfrom flask_cors import CORS"
)

# Enable CORS after app = Flask(__name__)
code = code.replace(
    "app = Flask(__name__)",
    "app = Flask(__name__)\nCORS(app)"
)

backend_path.write_text(code)
print("CORS added successfully.")


In [ ]:
!pip install -q flask-cors


In [ ]:
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!sudo apt-get install -y nodejs
!node -v
!npm -v


In [ ]:
# STEP A — install backend dependencies (CORRECT SYNTAX)

!pip install -q flask flask-cors gTTS

# ensure ffmpeg is installed
!ffmpeg -version || (sudo apt-get update -y && sudo apt-get install -y ffmpeg)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls -l "/content/drive/My Drive/AI-Automation" | sed -n '1,200p'


In [ ]:
%%bash
set -e

UI_SRC="/content/drive/My Drive/AI-Automation/kling_ai_react_ui.jsx"
APP_DIR="/content/kling-ui"

echo "=== STEP 0: check UI file exists ==="
if [ ! -f "${UI_SRC}" ]; then
  echo "ERROR: UI file not found at ${UI_SRC}"
  echo "Files in the folder:"
  ls -l "$(dirname "${UI_SRC}")"
  exit 2
fi
echo "Found UI file: ${UI_SRC}"
echo

# 1) create React app if missing (idempotent)
if [ ! -d "${APP_DIR}" ]; then
  echo "Creating React app (may take ~2min)..."
  npx create-react-app "${APP_DIR}"
else
  echo "React app already exists at ${APP_DIR}"
fi

# 2) ensure src exists & copy UI from Drive into src
mkdir -p "${APP_DIR}/src"
cp "${UI_SRC}" "${APP_DIR}/src/KlingAIUI.jsx"
echo "Copied UI -> ${APP_DIR}/src/KlingAIUI.jsx"

# 3) ensure App.js mounts KlingAIUI
cat > "${APP_DIR}/src/App.js" <<'EOF'
import React from "react";
import KlingAIUI from "./KlingAIUI";
export default function App(){ return <KlingAIUI />; }
EOF
echo "Wrote App.js to mount KlingAIUI"

# 4) clean formatting (remove CRLF/BOM if present)
sed -i 's/\r$//' "${APP_DIR}/src/KlingAIUI.jsx" || true
sed -i '1s/^\xEF\xBB\xBF//' "${APP_DIR}/src/KlingAIUI.jsx" || true
echo "Cleaned JSX formatting"

# 5) install deps (non-fatal if already installed)
cd "${APP_DIR}"
npm install framer-motion lucide-react --legacy-peer-deps || true

# 6) start dev server in background and show recent log
pkill -f "react-scripts start" || true
nohup npm start > /content/react_dev.log 2>&1 & echo $! > /content/react_dev.pid
sleep 6
echo "---- /content/react_dev.log (tail) ----"
tail -n 80 /content/react_dev.log || true
echo
echo "If you see 'Local: http://localhost:3000' then the dev server is running inside Colab."
echo "If not, paste the exact tail above here and I will fix the error."


In [ ]:
%%bash
cd /content/kling-ui
pkill -f "react-scripts start" || true
nohup npm start > /content/react_dev.log 2>&1 & echo $! > /content/react_dev.pid
sleep 4
tail -n 80 /content/react_dev.log


In [ ]:
%%bash

# 1) show what's using port 3000
echo "=== WHAT'S USING PORT 3000 ==="
ss -ltnp | grep ':3000' || echo "No process listening on :3000"

echo
echo "=== LIST REACT/Node PROCS ==="
ps aux | egrep 'node|react-scripts' | egrep -v 'grep' || true
echo

# 2) Kill any existing CRA dev server
echo "Attempting pkill -f react-scripts..."
pkill -f "react-scripts start" || true

# 3) Kill anything still occupying port 3000
PIDS=$(ss -ltnp | grep ':3000' | sed -E 's/.*pid=([0-9]+),.*/\1/' | tr '\n' ' ')
if [ -n "$PIDS" ]; then
  echo "Killing PIDs on :3000 -> $PIDS"
  kill -9 $PIDS || true
else
  echo "No PIDs found for :3000"
fi

sleep 1

# 4) confirm port is free
echo
echo "=== VERIFY PORT 3000 FREED ==="
ss -ltnp | grep ':3000' || echo "Port 3000 is free"

# 5) restart dev server
cd /content/kling-ui || { echo "ERROR: /content/kling-ui not found"; exit 1; }

echo "Starting React dev server (background). Log -> /content/react_dev.log"

nohup bash -lc "PORT=3000 npm start" > /content/react_dev.log 2>&1 & echo $! > /content/react_dev.pid

sleep 4

echo
echo "=== react_dev.log (tail) ==="
tail -n 60 /content/react_dev.log || true


In [ ]:
!curl -sS --max-time 5 https://advertizable-interpenetratively-abbie.ngrok-free.dev/ || echo "root failed"


In [ ]:
%%bash
set -e
UI_PATH="/content/kling-ui/src/KlingAIUI.jsx"
APP_DIR="/content/kling-ui"
LOG="/content/react_dev.log"

echo "1) Ensure app dir exists..."
if [ ! -d "$APP_DIR" ]; then
  echo "ERROR: React app not found at $APP_DIR. Create it first or change APP_DIR."
  exit 2
fi

echo "2) Installing UI deps (framer-motion, lucide-react) if missing..."
cd "$APP_DIR"
# legacy-peer-deps to avoid peer issues in Colab environment
npm install framer-motion lucide-react --legacy-peer-deps || true

echo "3) Writing revised KlingAIUI.jsx -> $UI_PATH (overwrite)"
cat > "$UI_PATH" <<'JSX'
/*
KlingAI — Advanced React UI (single-file component)
Revised to use real backend hooks and file upload handling.
Replace API_BASE with your backend public URL if different.
*/
import React, { useState, useRef } from 'react'
import { motion, AnimatePresence } from 'framer-motion'
import { CloudUpload, Play, Download, Settings, Trash2, FilePlus } from 'lucide-react'

// ---------- NETWORK / BACKEND HOOKS (real) ----------
const API_BASE = "https://advertizable-interpenetratively-abbie.ngrok-free.dev"; // <- set your backend public URL here

// Real generate: POST form (prompt, mode, duration, file optional) => { jobId }
async function realGenerate({ prompt, mode, duration, file }) {
  try {
    const fd = new FormData();
    fd.append("prompt", prompt || "");
    fd.append("mode", mode || "TEXT");
    fd.append("duration", String(duration || 4));
    if (file) fd.append("file", file, file.name || "upload.mp4");

    const res = await fetch(`${API_BASE}/api/generate`, {
      method: "POST",
      body: fd,
    });

    if (!res.ok) {
      const txt = await res.text();
      throw new Error(`Generate error: ${res.status} ${txt}`);
    }
    const json = await res.json();
    return json; // expect { jobId: "job-..." }
  } catch (err) {
    console.error("realGenerate error:", err);
    throw err;
  }
}

// Poll job status: calls onUpdate({ status, progress, logs, outputUrl })
// Returns stop() function to cancel polling
function pollJob(jobId, onUpdate, intervalMs = 1500) {
  let stopped = false;
  async function tick() {
    if (stopped) return;
    try {
      const r = await fetch(`${API_BASE}/api/job/${jobId}`);
      if (!r.ok) {
        const txt = await r.text();
        onUpdate({ status: "error", progress: 0, logs: [`HTTP ${r.status}: ${txt}`] });
        return;
      }
      const j = await r.json();
      let outUrl = j.outputUrl;
      if (outUrl && outUrl.startsWith("/")) outUrl = API_BASE + outUrl;
      onUpdate({ status: j.status, progress: j.progress || 0, logs: j.logs || [], outputUrl: outUrl });
      if (j.status === "done" || j.status === "error") {
        stopped = true;
        return;
      }
    } catch (e) {
      console.error("poll error", e);
      onUpdate({ status: "error", progress: 0, logs: [`poll error: ${String(e)}`] });
      stopped = true;
    }
    if (!stopped) setTimeout(tick, intervalMs);
  }
  setTimeout(tick, 200);
  return () => { stopped = true; };
}

// --- Presets storage ---
const defaultPresets = [
  { id: 'p1', name: 'Cute Tiny Animal', prompt: 'tiny playful kitten in a wool sweater, cinematic closeup', duration: 15 },
  { id: 'p2', name: 'Futuristic City', prompt: 'neon cyberpunk alley, rain, cinematic 3d', duration: 20 },
  { id: 'p3', name: 'Calm Nature Loop', prompt: 'tiny bird drinking water, soft light, loop', duration: 10 }
]

export default function KlingAIUI() {
  const [mode, setMode] = useState('TEXT')
  const [prompt, setPrompt] = useState('woolen cat playing')
  const [duration, setDuration] = useState(15)
  const [presets] = useState(defaultPresets)
  const [jobs, setJobs] = useState([]) // { jobId, prompt, progress, status, outputUrl, logs }
  const [selectedJob, setSelectedJob] = useState(null)
  const [uploads, setUploads] = useState([]) // uploaded sample list => {id,name,url,size,file}
  const [isGenerating, setIsGenerating] = useState(false)
  const [themeAccent, setThemeAccent] = useState('#06b6d4')
  const logsRef = useRef([])

  // create job using real backend
  const startGenerate = async () => {
    setIsGenerating(true)
    const cfg = { prompt, mode, duration }

    let fileObj = null
    if (uploads && uploads.length > 0 && uploads[0].file instanceof File) {
      fileObj = uploads[0].file
    }

    try {
      const json = await realGenerate({ prompt, mode, duration, file: fileObj })
      if (!json || !json.jobId) throw new Error("No jobId returned from backend")
      const jobId = json.jobId
      const newJob = { jobId, prompt, progress: 0, status: 'queued', outputUrl: null, logs: ['Job queued'] }
      setJobs(prev => [newJob, ...prev])
      setSelectedJob(jobId)

      const stop = pollJob(jobId, (update) => {
        setJobs(prev => prev.map(j => j.jobId===jobId ? { ...j, ...update } : j))
        if (update.status === 'done' || update.status === 'error') setIsGenerating(false)
      }, 1500)

      return () => stop()
    } catch (e) {
      setIsGenerating(false)
      console.error('Generate failed', e)
      setJobs(prev => [{ jobId: 'local-error', prompt, progress: 0, status: 'error', logs: [String(e)], outputUrl: null }, ...prev])
    }
  }

  // upload manager: store File object and blob url for preview
  const onFileUpload = (file) => {
    const id = 's' + Math.random().toString(36).slice(2,8)
    const url = URL.createObjectURL(file)
    const item = { id, name: file.name, url, size: file.size, file }
    setUploads(u => [item, ...u])
  }

  const removeUpload = (id) => setUploads(u => u.filter(x => x.id !== id))

  // UI helpers
  const activeJob = jobs.find(j => j.jobId === selectedJob) || jobs[0] || null

  return (
    <div className="min-h-screen bg-gradient-to-b from-slate-900 to-[#041018] text-slate-100 p-6">
      <div className="max-w-6xl mx-auto">
        <header className="flex items-center justify-between mb-6">
          <div className="flex items-center gap-4">
            <motion.div whileHover={{ scale: 1.05 }} className="p-3 rounded-2xl bg-gradient-to-br from-[#042b38] to-[#071124] shadow-2xl">
              <img src="/logo192.png" alt="KlingAI" className="w-10 h-10" />
            </motion.div>
            <div>
              <h1 className="text-2xl font-extrabold tracking-tight">KlingAI — TinyShort Lab</h1>
              <p className="text-sm text-slate-300">Advanced UI • Multi-preview • Live logs • Presets • Upload manager</p>
            </div>
          </div>
          <div className="flex items-center gap-3">
            <button className="px-3 py-2 rounded-lg bg-slate-800/50 hover:bg-slate-800/70 flex items-center gap-2"><Settings size={16} />Settings</button>
            <button onClick={() => { setThemeAccent('#06b6d4') }} className="px-3 py-2 rounded-lg bg-[#06b6d4]/20 hover:bg-[#06b6d4]/30">Teal</button>
            <button onClick={() => { setThemeAccent('#f97316') }} className="px-3 py-2 rounded-lg bg-[#f97316]/20 hover:bg-[#f97316]/30">Orange</button>
          </div>
        </header>

        <main className="grid grid-cols-12 gap-6">
          {/* Left column: controls */}
          <section className="col-span-4 bg-[#071124] p-4 rounded-2xl shadow-md">
            <div className="mb-4">
              <label className="text-xs text-slate-400">Mode</label>
              <div className="mt-2 flex gap-2">
                <button onClick={() => setMode('TEXT')} className={`px-3 py-2 rounded-md ${mode==='TEXT' ? 'bg-slate-700' : 'bg-slate-800/40'}`}>Text</button>
                <button onClick={() => setMode('IMAGE')} className={`px-3 py-2 rounded-md ${mode==='IMAGE' ? 'bg-slate-700' : 'bg-slate-800/40'}`}>Image</button>
              </div>
            </div>

            <div className="mb-3">
              <label className="text-xs text-slate-400">Prompt</label>
              <textarea value={prompt} onChange={e=>setPrompt(e.target.value)} rows={3} className="mt-2 w-full rounded-md bg-slate-900 p-3 text-sm" />
            </div>

            <div className="mb-4 flex items-center gap-2">
              <label className="text-xs text-slate-400">Duration (sec)</label>
              <input type="number" value={duration} onChange={e=>setDuration(Number(e.target.value))} min={1} max={60} className="ml-auto w-24 p-2 rounded-md bg-slate-900 text-sm" />
            </div>

            <div className="flex gap-3">
              <button onClick={startGenerate} disabled={isGenerating} className="flex-1 py-3 rounded-xl bg-gradient-to-r from-[#06b6d4] to-[#0ea5a4] font-bold shadow-lg flex items-center justify-center gap-2"><Play size={18}/>Generate</button>
              <button className="py-3 px-4 rounded-xl bg-slate-700/40 flex items-center gap-2"><CloudUpload size={16}/>Upload</button>
            </div>

            <div className="mt-6">
              <h3 className="text-sm text-slate-300 mb-2">Presets</h3>
              <div className="flex flex-col gap-2">
                {presets.map(p => (
                  <motion.button key={p.id} whileHover={{ scale: 1.02 }} onClick={() => { setPrompt(p.prompt); setDuration(p.duration) }} className="text-left p-3 rounded-lg bg-slate-800/40">
                    <div className="flex justify-between items-center">
                      <div>
                        <div className="font-semibold">{p.name}</div>
                        <div className="text-xs text-slate-400">{p.duration}s</div>
                      </div>
                      <div className="text-xs text-slate-400">Apply</div>
                    </div>
                  </motion.button>
                ))}
              </div>
            </div>

            <div className="mt-6">
              <h3 className="text-sm text-slate-300 mb-2">Upload Manager</h3>
              <div className="flex gap-2 items-center">
                <input id="file-in" type="file" accept="video/*" onChange={e => e.target.files && onFileUpload(e.target.files[0])} className="hidden" />
                <label htmlFor="file-in" className="cursor-pointer px-3 py-2 rounded-lg bg-slate-800/40 flex items-center gap-2"><FilePlus size={14}/>Add Sample</label>
              </div>
              <div className="mt-3 flex flex-col gap-2 max-h-36 overflow-auto">
                {uploads.length===0 && <div className="text-xs text-slate-500">No uploads yet</div>}
                {uploads.map(u => (
                  <div key={u.id} className="flex items-center justify-between p-2 bg-slate-900 rounded">
                    <div className="truncate text-sm">{u.name}</div>
                    <div className="flex items-center gap-2">
                      <a href={u.url} target="_blank" rel="noreferrer" className="text-xs text-slate-300 underline">Preview</a>
                      <button onClick={() => removeUpload(u.id)} className="p-1 rounded bg-slate-800/50"><Trash2 size={12}/></button>
                    </div>
                  </div>
                ))}
              </div>
            </div>
          </section>

          {/* Right column: preview and job list */}
          <section className="col-span-8">
            <div className="grid grid-cols-12 gap-4">
              <div className="col-span-8 bg-[#071124] rounded-2xl p-4 shadow-md">
                <div className="flex items-center justify-between mb-3">
                  <div>
                    <h2 className="font-bold text-lg">Preview</h2>
                    <div className="text-xs text-slate-400">Main preview — click thumbnails to swap.</div>
                  </div>
                  <div className="flex items-center gap-2">
                    <button onClick={() => window.open(activeJob?.outputUrl || '#')} className="px-3 py-2 rounded bg-slate-800/50"><Download size={16}/>Download</button>
                  </div>
                </div>

                <div className="bg-black rounded-lg overflow-hidden relative" style={{ height: 480 }}>
                  <AnimatePresence>
                    {activeJob && activeJob.outputUrl ? (
                      <motion.video key={activeJob.jobId} src={activeJob.outputUrl} controls autoPlay style={{ width: '100%', height: '100%', objectFit: 'cover' }} />
                    ) : (
                      <motion.div key="placeholder" initial={{ opacity: 0 }} animate={{ opacity: 1 }} exit={{ opacity: 0 }} className="w-full h-full flex items-center justify-center text-slate-400">
                        <div className="text-center">
                          <div className="mb-2 text-sm">No generated output yet</div>
                          <div className="text-xs">Press Generate to start a job. A created fallback will be used if no sample provided.</div>
                        </div>
                      </motion.div>
                    )}
                  </AnimatePresence>
                </div>

                <div className="mt-3 grid grid-cols-3 gap-2">
                  {jobs.slice(0,3).map(j => (
                    <motion.div key={j.jobId} whileHover={{ scale: 1.02 }} onClick={() => setSelectedJob(j.jobId)} className={`p-2 rounded-lg ${selectedJob===j.jobId ? 'ring-2 ring-slate-600' : 'bg-slate-900/40'}`}>
                      <div className="h-28 bg-black rounded mb-2 flex items-center justify-center text-slate-500">{j.outputUrl ? <video src={j.outputUrl} style={{ width: '100%', height: '100%', objectFit: 'cover' }} /> : 'Preview'}</div>
                      <div className="text-xs">{j.prompt.slice(0,40)}</div>
                      <div className="text-2xs text-slate-400">{j.progress}% — {j.status}</div>
                    </motion.div>
                  ))}
                </div>

              </div>

              <div className="col-span-4 bg-[#071124] rounded-2xl p-4 shadow-md flex flex-col">
                <h3 className="font-semibold mb-2">Jobs & Logs</h3>
                <div className="flex-1 overflow-auto p-2 bg-slate-900/30 rounded">
                  {jobs.length===0 && <div className="text-sm text-slate-500">No jobs yet — generate one.</div>}
                  {jobs.map(j => (
                    <div key={j.jobId} className="p-2 border-b border-slate-800 last:border-b-0">
                      <div className="flex justify-between items-center">
                        <div>
                          <div className="font-medium">{j.jobId}</div>
                          <div className="text-xs text-slate-400">{j.prompt.slice(0,60)}</div>
                        </div>
                        <div className="text-sm">{j.progress}%</div>
                      </div>
                      <div className="mt-2 text-xs text-slate-400 max-h-20 overflow-auto">
                        {(j.logs||[]).slice(-5).map((l, i) => <div key={i}>{l}</div>)}
                      </div>
                      <div className="mt-2 flex gap-2">
                        <button onClick={() => { setSelectedJob(j.jobId) }} className="text-xs px-2 py-1 rounded bg-slate-800/40">View</button>
                        <button onClick={() => setJobs(prev => prev.filter(x => x.jobId !== j.jobId))} className="text-xs px-2 py-1 rounded bg-red-700/60">Delete</button>
                      </div>
                    </div>
                  ))}
                </div>

                <div className="mt-3 text-xs text-slate-500">Live logs appear here while jobs run.</div>
              </div>
            </div>
          </section>
        </main>

        <footer className="mt-8 text-center text-slate-500 text-sm">Made with ❤️ for AI Creative Explorer — adapt backend endpoints to hook gen model.</footer>
      </div>
    </div>
  )
}
JSX

echo "4) Kill any existing react dev server on port 3000 and restart dev server in background..."
# kill previous react-scripts starts on port 3000
pkill -f "react-scripts start" || true
# start dev server in background and log
NOHUP_LOG="$LOG"
nohup npm start > "$NOHUP_LOG" 2>&1 &
sleep 2

echo "5) Show tail of dev log (last 60 lines)"
sleep 2
tail -n 60 "$LOG" || true

echo "PATCH COMPLETE. If React dev server shows 'Local: http://localhost:3000' the app is running in this Colab. Open it via your tunnel/ngrok or forward port."


In [ ]:
# install pyngrok (quiet)
!pip install -q pyngrok


In [ ]:
# verify import (run in Python cell)
try:
    import pyngrok
    from pyngrok import ngrok
    print("pyngrok import OK —", pyngrok.__version__)
except Exception as e:
    print("pyngrok import FAILED:", repr(e))
    raise


In [ ]:
from getpass import getpass
from pyngrok import ngrok, conf

token = getpass("Paste ngrok authtoken (won't echo): ").strip()
if not token:
    raise SystemExit("No token provided - stopping.")
# configure token
conf.get_default().auth_token = token
ngrok.set_auth_token(token)

# close previous tunnels safely
try:
    for t in ngrok.get_tunnels():
        try: ngrok.disconnect(t.public_url)
        except Exception: pass
except Exception:
    pass

# open new tunnel to your Flask backend (port 7860)
tunnel = ngrok.connect(7860, "http")
print("NGROK PUBLIC URL ->", tunnel.public_url)
print("Active tunnels:", ngrok.get_tunnels())


In [ ]:
FILE = "/content/drive/My Drive/AI-Automation/sample.mp4"


In [ ]:
import requests, os, time

NGROK = None
try:
    NGROK = [t.public_url for t in __import__('pyngrok').ngrok.get_tunnels()][0]
except Exception:
    # fallback to last-known public URL you shared earlier
    NGROK = "https://advertizable-interpenetratively-abbie.ngrok-free.dev"

print("Checking", NGROK + "/")
try:
    r = requests.get(NGROK + "/", timeout=6)
    print("ROOT status:", r.status_code)
    print(r.text[:600])
except Exception as e:
    print("ROOT check failed:", repr(e))


In [ ]:
# Robust: auto-pick sample + POST -> /api/generate, poll job, download output
# Paste & run in a Python cell in Colab.
import requests, os, time, glob, json, sys
from pathlib import Path

# === CONFIG ===
NGROK = "https://advertizable-interpenetratively-abbie.ngrok-free.dev"  # <-- set your public backend URL here
OUT_FILE = "/content/generated_output.mp4"
POLL_INTERVAL = 2.0        # seconds
POLL_TIMEOUT = 300.0       # total seconds to wait for job to finish
POST_TIMEOUT = 120.0       # seconds for the POST request

# Candidate sample paths (session history + sensible fallbacks)
candidates = [
    "/content/drive/My Drive/AI-Automation/sample.mp4",     # <- your Drive sample (preferred)
    "/content/sample_from_upload.mp4",
    "/content/sample.mp4",
    "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4",
]

# add up to 12 mp4 files found under /content and /mnt as extra fallbacks
for root in ("/content", "/mnt"):
    if os.path.isdir(root):
        found = glob.glob(os.path.join(root, "**", "*.mp4"), recursive=True)
        for p in found[:12]:
            if p not in candidates:
                candidates.append(p)

# pick first existing candidate
sample = next((p for p in candidates if os.path.isfile(p)), None)

if not sample:
    # show helpful diagnostics
    print("Tried candidate paths (none found):")
    for p in candidates:
        print("  ", p)
    print("\nQuick scan under /content (top 8 mp4s):")
    found = glob.glob("/content/**/*.mp4", recursive=True)[:8]
    if found:
        for p in found:
            print("   ", p)
    else:
        print("   (no mp4s found under /content)")
    raise SystemExit("❌ Sample missing: please upload an mp4 to Drive or /content and re-run the cell.")

print("Using sample:", sample)
gen_url = NGROK.rstrip("/") + "/api/generate"
print("Posting to", gen_url)

# POST file
try:
    with open(sample, "rb") as fh:
        files = {"file": ("sample.mp4", fh, "video/mp4")}
        data = {
            "prompt": "9:16 tiny mechanical fox exploring a sunlit garden, cinematic, no logos",
            "mode": "IMAGE",
            "duration": "4"
        }
        resp = requests.post(gen_url, data=data, files=files, timeout=POST_TIMEOUT)
except requests.exceptions.RequestException as e:
    raise SystemExit(f"POST to backend failed: {e}")

print("HTTP status:", resp.status_code)
try:
    j = resp.json()
except Exception:
    print("Non-JSON response text (trim):", resp.text[:1000])
    raise SystemExit("Backend didn't return JSON.")

if "jobId" not in j:
    print("Response JSON:", json.dumps(j, indent=2))
    raise SystemExit("Backend did not return jobId. Check backend logs/endpoint.")

jobid = j["jobId"]
print("Created jobId:", jobid)

# Poll job
job_url = NGROK.rstrip("/") + f"/api/job/{jobid}"
start = time.time()
print("Polling", job_url)
last_status = None
while True:
    try:
        r = requests.get(job_url, timeout=30)
    except requests.exceptions.RequestException as e:
        print("Poll error:", e)
        elapsed = time.time() - start
        if elapsed > POLL_TIMEOUT:
            raise SystemExit("Polling timeout (network flaky).")
        time.sleep(2.0)
        continue

    try:
        js = r.json()
    except Exception:
        print("Non-JSON polling response (trim):", r.text[:800])
        raise SystemExit("Polling endpoint returned non-JSON.")

    status = js.get("status")
    progress = js.get("progress", 0)
    logs = js.get("logs", [])
    outputUrl = js.get("outputUrl")

    if status != last_status:
        print(f"[{int(time.time()-start)}s] STATUS:", status, "| progress:", progress)
        if logs:
            print(" LOGS (tail):", logs[-3:])
        last_status = status
    else:
        # minimize spam, only show progress steps
        print(f"[{int(time.time()-start)}s] progress: {progress}% (status={status})", end="\r")

    if status == "done" or status == "error":
        break

    if time.time() - start > POLL_TIMEOUT:
        raise SystemExit("Polling timed out. Increase POLL_TIMEOUT or check backend.")

    time.sleep(POLL_INTERVAL)

# final status handling
print("\nFINAL STATUS:", status)
if status == "error":
    print("Logs:", logs)
    raise SystemExit("Job ended with error. Check backend logs.")

# Download output if URL provided
if not outputUrl:
    print("No outputUrl returned in job response. Check backend 'output' path. Logs:", logs)
    raise SystemExit("No output to download.")

# normalize outputUrl (if backend returns relative path)
if outputUrl.startswith("/"):
    outputUrl = NGROK.rstrip("/") + outputUrl

print("Downloading output from:", outputUrl)
try:
    r = requests.get(outputUrl, stream=True, timeout=120)
    r.raise_for_status()
    total = 0
    with open(OUT_FILE, "wb") as fw:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                fw.write(chunk)
                total += len(chunk)
    print("Saved video to:", OUT_FILE, " (bytes:", total, ")")
except requests.exceptions.RequestException as e:
    raise SystemExit(f"Failed to download output: {e}")

print("✅ Done. If you want, paste the full output path and I'll patch the React UI to point to it.")


In [ ]:
import requests, time, os
NGROK = "https://advertizable-interpenetratively-abbie.ngrok-free.dev"  # change if different
JOBID = "job-xxxx"  # <-- replace with jobId returned above
poll_url = NGROK + f"/api/job/{JOBID}"
out_url = NGROK + f"/api/output/{JOBID}"

for _ in range(40):
    r = requests.get(poll_url, timeout=10)
    if r.status_code != 200:
        print("poll status", r.status_code, r.text[:400]); break
    js = r.json()
    print("STATUS:", js.get("status"), "| PROGRESS:", js.get("progress"))
    if js.get("status") == "done":
        print("Output URL ->", js.get("outputUrl") or out_url)
        break
    time.sleep(1.5)

# download if done:
if js.get("status") == "done":
    r = requests.get(out_url, stream=True, timeout=60)
    target = "/content/generated_output.mp4"
    with open(target, "wb") as f:
        for chunk in r.iter_content(1024*1024):
            if chunk: f.write(chunk)
    print("Saved video to:", target)


In [63]:
!pip install -r "/content/drive/My Drive/AI-Automation/requirements.txt"

/content/drive/My Drive/AI-Automation/kling_ai_react_ui.jsx

In [88]:
# server.py
from flask import Flask, request, jsonify
import threading, subprocess, uuid, os, json, time

app = Flask(__name__)
WORKDIR = os.path.abspath('work'); os.makedirs(WORKDIR, exist_ok=True)
jobs = {}

def generate_call(jobid, prompt, mode, duration, seed_url=None):
    jobs[jobid]['status'] = 'running'; jobs[jobid]['logs'].append('starting generation')
    cmd = ['python3','generate_video.py','--prompt', prompt, '--duration', str(duration), '--outdir', WORKDIR]
    if seed_url:
        cmd += ['--seed_url', seed_url]
    try:
        p = subprocess.run(cmd, capture_output=True, text=True, timeout=60*60)
        jobs[jobid]['logs'].append(p.stdout.strip())
        if p.returncode == 0:
            # try parse last JSON line for {"output": "<path>"}
            try:
                out = json.loads(p.stdout.strip().splitlines()[-1])['output']
                jobs[jobid]['status'] = 'done'; jobs[jobid]['outputUrl'] = out; jobs[jobid]['progress'] = 100
                jobs[jobid]['logs'].append('finished: ' + str(out))
            except Exception as e:
                jobs[jobid]['status'] = 'error'
                jobs[jobid]['logs'].append('parse error: ' + repr(e))
        else:
            jobs[jobid]['status'] = 'error'
            jobs[jobid]['logs'].append('generator failed. rc=%s stderr=%s' % (p.returncode, p.stderr[:200]))
    except Exception as e:
        jobs[jobid]['status'] = 'error'
        jobs[jobid]['logs'].append('exception: ' + repr(e))

@app.route('/api/generate', methods=['POST'])
def api_generate():
    data = request.json or request.form.to_dict()
    prompt = data.get('prompt','tiny glowing fox')
    mode = data.get('mode','TEXT')
    duration = int(data.get('duration') or 15)
    seed_url = data.get('seed_url')
    jobid = 'job-' + uuid.uuid4().hex[:8]
    jobs[jobid] = {'jobId':jobid,'status':'queued','progress':0,'logs':['queued'],'outputUrl':None}
    threading.Thread(target=generate_call, args=(jobid,prompt,mode,duration,seed_url), daemon=True).start()
    return jsonify({'jobId': jobid}), 202

@app.route('/api/job/<jobid>', methods=['GET'])
def api_job(jobid):
    return jsonify(jobs.get(jobid, {'status':'notfound'}))

if __name__ == '__main__':
    # debug mode fine for local/Grok/Colab testing
    app.run(host='0.0.0.0', port=8787, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8787
 * Running on http://172.28.0.12:8787
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764062619
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_092456
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764062920
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_093008
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764063220
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_093522
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764063521
[AUTO-PERSIST] Removed old backup: /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764059607
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_094039
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764

<IPython.core.display.Javascript object>

[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_100139
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764065026
[AUTO-PERSIST] Removed old backup: /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764061414
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_100652
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764065327
[AUTO-PERSIST] Removed old backup: /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764061716
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_101207
[AUTO-PERSIST] [autosave] snapshot -> /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764065627
[AUTO-PERSIST] Removed old backup: /content/drive/MyDrive/AI-Automation/session_backups/snapshot_1764062017
[AUTOSAVE] Backup created at /content/drive/MyDrive/YT_Backups/backup_20251125_101721

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8787
 * Running on http://172.28.0.12:8787
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
